## RAG using Langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API-KEY"

# Install Libraries

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
                faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate


### Step 1-a.  Indexing (Document Ingestion)

In [6]:
video_id = '5VMckmQneCk' # only the ID not full URL

try:
  #if you don't care which language, this returns the "best" one
  transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

  # Flatten it to plain text
  transcript = " ".join(chunk["text"] for chunk in transcript_list)
  print(transcript)

except TranscriptsDisabled:
  print("No caption available for this video."),

the Galaxy s25 Ultra is the most enticing phone that Samsung has ever made the thing is addictively delightful to use I'd say this is the closest phone has ever come to being an iPhone killer but at the same time should you actually go out and buy one right now in most cases I would say no let me explain why so on one hand it's actually kind of hard to describe just how much I love using this thing I mean just from a technical materials perspective the glass being used on the front and back the now stronger Titanium on the body this is one tough phone I've been specifically using it without a case to test it and so far at least it's been in a camera bag it's been in backpacks pockets it's been embedded into our table during a failed intro I even painfully sat here rubbing my wedding ring against it and there is a little more dust collection than I would like but zero in the way of damage only thing that has unfortunately marked it is a pretty nasty meter and a half drop straight onto c

In [7]:
transcript_list

[{'text': 'the Galaxy s25 Ultra is the most',
  'start': 0.04,
  'duration': 3.919},
 {'text': 'enticing phone that Samsung has ever',
  'start': 1.959,
  'duration': 4.041},
 {'text': 'made the thing is addictively delightful',
  'start': 3.959,
  'duration': 3.761},
 {'text': "to use I'd say this is the closest phone",
  'start': 6.0,
  'duration': 3.96},
 {'text': 'has ever come to being an iPhone killer',
  'start': 7.72,
  'duration': 4.039},
 {'text': 'but at the same time should you actually',
  'start': 9.96,
  'duration': 3.52},
 {'text': 'go out and buy one right now in most',
  'start': 11.759,
  'duration': 4.241},
 {'text': 'cases I would say no let me explain why',
  'start': 13.48,
  'duration': 4.52},
 {'text': "so on one hand it's actually kind of",
  'start': 16.0,
  'duration': 3.84},
 {'text': 'hard to describe just how much I love',
  'start': 18.0,
  'duration': 3.519},
 {'text': 'using this thing I mean just from a',
  'start': 19.84,
  'duration': 3.4},
 {'text'

### Step 1.b Text Splitting

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)

chunks = splitter.create_documents([transcript])

In [11]:
len(chunks)

48

In [12]:
chunks[1]

Document(metadata={}, page_content="I mean just from a technical materials perspective the glass being used on the front and back the now stronger Titanium on the body this is one tough phone I've been specifically using it without a case to test it and so far at least it's been in a camera bag it's been in backpacks pockets it's been embedded into our table during a failed intro I even painfully sat here rubbing my wedding ring against it and there is a little more dust collection than I would like but zero in the way of damage")

### Step 1.c & 1.d - Indexing
## (Embedding Generation and Storing in Vector Store)

In [13]:
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embedding)

In [14]:
vector_store.index_to_docstore_id

{0: '58aef6b1-76cd-4a88-bbd3-6fe88c079d2a',
 1: '90331432-9598-44ed-a8a0-d93448a31bfe',
 2: 'a25868d4-86e8-477e-9283-0768dd42c4b3',
 3: 'cc729b7c-4a25-4fb9-8e78-9fab2be70f1c',
 4: 'f7349acd-d8a1-4fdc-bc09-2ef2e43947e8',
 5: '582a271e-70cc-407b-ad80-23c589929ef5',
 6: 'c8f261d8-4e3c-4661-982b-4d37146f8706',
 7: 'e9409093-e2d6-45b9-b11d-501df1bb91bf',
 8: 'c0c5e89a-1f59-4672-9af4-de5c2fc8de27',
 9: '166a3060-1831-41a5-afe4-8e238a194d38',
 10: 'f288cb6e-eccb-409a-ac0e-5803bba3cd4a',
 11: '9fbd733a-a995-43e4-b5df-24dff32ebc46',
 12: 'e3b67a13-a117-4838-a5b3-bae551993140',
 13: 'b584a83a-4b13-4996-a4bf-643a6f579b2f',
 14: '6d0f00c1-fe18-4830-af78-37db64e3a2b0',
 15: '8797e3f0-a3a3-4cac-891b-b12381c0b200',
 16: 'e88d8f25-4b95-4c88-b80c-83f8f0557c93',
 17: 'e4559ff2-3924-4a90-9524-b45070a72b12',
 18: '2a3a96a9-e98e-459d-8a7f-3b710b3595f9',
 19: '2d2af996-363d-4cf2-9ada-41c2ddc31e33',
 20: '19a11ea8-773a-4599-a019-30e657f7297f',
 21: '2976c3ab-ed77-4cdf-a1ca-62aac29dbfe4',
 22: '7c364675-d299-

In [15]:
vector_store.get_by_ids(['3afdd03f-52a5-4a79-b61b-a897da5749db'])

[Document(id='3afdd03f-52a5-4a79-b61b-a897da5749db', metadata={}, page_content="every single VPN has a cap you might be able to share your login with two people five people seven people with Sur shark VPN one subscription gets VPN access to you your buddies and your entire city if you want it to it's the most stable VPN I've used the app just works whether you're on your phone or your computer and it's one of the most useful Swiss Army knif tools to have in your back pocket it can make websites that were previously misbehaving just suddenly start working again it can let")]

## Step-2 Retrieval

In [19]:
retriever = vector_store.as_retriever(
    search_type = 'similarity',
    search_kwargs={"k": 3})

In [20]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7862fc1e04d0>, search_kwargs={'k': 3})

In [21]:
retriever.invoke("What's new in S25 Ultra?")

[Document(id='9fbd733a-a995-43e4-b5df-24dff32ebc46', metadata={}, page_content='new Android skin but you should be getting practically the full one ui7 feature set on the s24 ultra 2 this new phone has an even faster chip than last year like significantly faster just to give you an idea we ran a 30-minute stress test on this s25 Ultra the s24 ultra before it and the iPhone 16 Pro Max and while the s25 ultra does have the lowest stability meaning that over the course of that 30 minutes it lost the most performance compared to where it started because of how high its'),
 Document(id='e88d8f25-4b95-4c88-b80c-83f8f0557c93', metadata={}, page_content="software not hardware and then there is the AI part of the equation the majority of the appeal of this s25 Ultra more so than maybe any phone ever released is tied to AI we've been told this AI is only free till the end of 2025 but we still have no idea how much the AI is going to cost now I will say some of these new AI features are amazing l

## Augmentation

In [22]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [23]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided transcript context.
    If the context is insufficient, just say you don't know.

    {context}

    Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [24]:
question = "is the topic of iphone or ios discussed in this video? if yes then what is the topic?"

retrieved_docs = retriever.invoke(question)

In [25]:
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [26]:
retrieved_docs

[Document(id='582a271e-70cc-407b-ad80-23c589929ef5', metadata={}, page_content="the 5 years that we've been together but I can kind of see why she said it here it feels like this software is specifically designed to feel familiar to an iPhone user the icons and widgets are now more in line than ever the multitasking page is now practically identical the way that you can Now swipe down from the right to pull up your Control Center and then swipe down from the left to open a separate notifications tray the way that you customize your lock screen is like okay copy my homework"),
 Document(id='c0c5e89a-1f59-4672-9af4-de5c2fc8de27', metadata={}, page_content="understands the assignment every time people always talk about maximum brightness oh dear God but I really like how dim the minimum brightness on this gets it's less strenuous on your eyes just before bed and the thing is just so fast you know the iPhone always has this implicit reputation as the smooth one the one that does animations

In [27]:
final_prompt = prompt.invoke({"context" : context_text, "question" : question})

In [29]:
final_prompt

StringPromptValue(text="\n    You are a helpful assistant.\n    Answer only from the provided transcript context.\n    If the context is insufficient, just say you don't know.\n\n    the 5 years that we've been together but I can kind of see why she said it here it feels like this software is specifically designed to feel familiar to an iPhone user the icons and widgets are now more in line than ever the multitasking page is now practically identical the way that you can Now swipe down from the right to pull up your Control Center and then swipe down from the left to open a separate notifications tray the way that you customize your lock screen is like okay copy my homework\n\nunderstands the assignment every time people always talk about maximum brightness oh dear God but I really like how dim the minimum brightness on this gets it's less strenuous on your eyes just before bed and the thing is just so fast you know the iPhone always has this implicit reputation as the smooth one the o

## Step 4 - Generation

In [28]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of iPhone and iOS is discussed in this video. The discussion revolves around how the software of a Samsung device feels familiar to an iPhone user, highlighting similarities in design elements such as icons, widgets, multitasking, and the Control Center. The speaker also contrasts the performance and user experience of the Samsung device with that of the iPhone, noting that the Samsung feels fast and smooth, challenging the reputation of the iPhone for having superior animations.


## Building a Chain

In [30]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [31]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [32]:
parallel_chain = RunnableParallel(
    context = retriever | RunnableLambda(format_docs),
    question = RunnablePassthrough()
)

In [33]:
parallel_chain.invoke("What's new in S25 Ultra?")

{'context': "new Android skin but you should be getting practically the full one ui7 feature set on the s24 ultra 2 this new phone has an even faster chip than last year like significantly faster just to give you an idea we ran a 30-minute stress test on this s25 Ultra the s24 ultra before it and the iPhone 16 Pro Max and while the s25 ultra does have the lowest stability meaning that over the course of that 30 minutes it lost the most performance compared to where it started because of how high its\n\nsoftware not hardware and then there is the AI part of the equation the majority of the appeal of this s25 Ultra more so than maybe any phone ever released is tied to AI we've been told this AI is only free till the end of 2025 but we still have no idea how much the AI is going to cost now I will say some of these new AI features are amazing like AI select Game Changer you know when someone sends you a whole paragraph and you just want to copy one sentence from that paragraph This solves

In [34]:
parser = StrOutputParser()

In [35]:
main_chain = parallel_chain | prompt | llm | parser

In [36]:
main_chain.invoke("What's new in S25 Ultra?")

'The S25 Ultra features a new Android skin with practically the full One UI 7 feature set, a significantly faster chip compared to the previous model, and enhanced AI capabilities. Some of the new AI features include AI select, which allows users to easily copy specific sentences from longer paragraphs.'

In [38]:
main_chain.invoke("What are the key features of S25 Ultra?")

'The key features of the S25 Ultra include a new Android skin with practically the full One UI 7 feature set, a significantly faster chip compared to the previous model, and a strong emphasis on AI capabilities. Some notable AI features mentioned are the AI select function, which allows users to easily copy specific sentences from longer texts. Additionally, the materials used for the front and back glass are highlighted as a positive aspect of the phone.'

In [42]:
main_chain.invoke("summarize s25 ultra")

"The Galaxy S25 Ultra is described as Samsung's most enticing phone to date, featuring a significantly faster chip and a full One UI 7 feature set. It offers an addictively delightful user experience and is considered the closest competitor to the iPhone. However, despite its appeal, the recommendation is to hold off on purchasing it for now. A significant aspect of its appeal is tied to new AI features, which are currently free until the end of 2025, but the future cost of these features is uncertain."